In [54]:
import os

# Change the current working directory to the specified path
new_directory = r'C:\Users\vladimir.shpilkin\Desktop\Learning\Python-Projects\py_sql_analysis'
os.chdir(new_directory)

# Print the new current working directory
print('New directory is', os.getcwd())

# Import necessary libraries
import sqlite3
import pandas as pd

# Read the CSV file located in the new directory
bank_data = 'Employee_Salary_Dataset.csv'
bank_data_df = pd.read_csv(bank_data)

# Establish a connection to the SQLite database
connection_employee_salary = sqlite3.connect('Employee_Salary_Dataset.db')

# Write the DataFrame to the SQLite database
employee_salary = bank_data_df.to_sql('Employee_Salary_Dataset', connection_employee_salary, if_exists='replace', index=False)

# Create a cursor for the database
cursor = connection_employee_salary.cursor()


New directory is c:\Users\vladimir.shpilkin\Desktop\Learning\Python-Projects\py_sql_analysis


In [55]:
#find the average salary for male workers
results_male = cursor.execute('SELECT "Age", ROUND(AVG("Salary"), 1) AS "Avg_Male_Salary" FROM "Employee_Salary_Dataset" WHERE "Gender" = "Male"')
avg_male_salary = results_male.fetchone()[1]
print(avg_male_salary)

2063626.5


In [56]:
#find the average salary for female workers
results_female = cursor.execute('SELECT "Age", ROUND(AVG("Salary"), 1) AS "Avg_Male_Salary" FROM "Employee_Salary_Dataset" WHERE "Gender" = "Female"')
avg_female_salary = results_male.fetchone()[1]
print(avg_female_salary)

2054916.7


In [57]:
#find the difference
round(avg_male_salary-avg_female_salary,2)

8709.8

In [248]:
#find the average salary per age group and gender
age_bins=cursor.execute('''
                            SELECT CASE WHEN "Age" BETWEEN 18 and 24 then "18-24"
                            WHEN "Age" BETWEEN 25 and 34 then "25-34"
                            WHEN "Age" BETWEEN 35 and 44 then "35-44"
                            WHEN "Age" BETWEEN 45 and 54 then "45-54"
                            WHEN "Age" BETWEEN 55 and 65 then "55-65"
                            WHEN "Age" <18 then "underage"
                            WHEN "Age" >65 then "retirement"
                            END AS age_bin, round(avg("Salary"),1), "Gender",
                            case when Gender='Male' then count(ID) else 0 END AS male_count,
                            case when Gender='Female' then COUNT(ID) else 0 end as female_count
                            from Employee_Salary_Dataset
                            GROUP BY age_bin, "Gender"
                            '''
                            )

In [249]:
#display the avg. salary per gender and age bin
age_bins.fetchall()

[('18-24', 36666.7, 'Female', 0, 6),
 ('18-24', 20100.0, 'Male', 5, 0),
 ('25-34', 379250.0, 'Female', 0, 4),
 ('25-34', 2087500.0, 'Male', 4, 0),
 ('35-44', 61500.0, 'Female', 0, 1),
 ('35-44', 2183383.3, 'Male', 3, 0),
 ('45-54', 4730000.0, 'Female', 0, 5),
 ('45-54', 5023333.3, 'Male', 3, 0),
 ('55-65', 5770000.0, 'Female', 0, 2),
 ('55-65', 5001000.0, 'Male', 1, 0),
 ('underage', 10000.0, 'Male', 1, 0)]

In [221]:
#find the average salary per age group and gender
age_bins_detailed=cursor.execute('''
                            SELECT
    CASE
        WHEN Age BETWEEN 18 AND 24 THEN '18-24'
        WHEN Age BETWEEN 25 AND 34 THEN '25-34'
        WHEN Age BETWEEN 35 AND 44 THEN '35-44'
        WHEN Age BETWEEN 45 AND 54 THEN '45-54'
        WHEN Age BETWEEN 55 AND 65 THEN '55-65'
        WHEN Age < 18 THEN 'underage'
        WHEN Age > 65 THEN 'retirement'
    END AS age_bin,
    "Gender",
    ROUND(
        (AVG(CASE WHEN "Gender" = 'Male' THEN Salary else NULL END) - AVG(CASE WHEN "Gender" = 'Female' THEN Salary else NULL END)),
        1
    ) AS salary_diff_male_from_female
FROM Employee_Salary_Dataset
group by age_bin
ORDER BY age_bin,salary_diff_male_from_female;


                            '''
                            )

In [222]:
age_bins_detailed.fetchall()


[('18-24', 'Male', -16566.7),
 ('25-34', 'Female', 1708250.0),
 ('35-44', 'Female', 2121883.3),
 ('45-54', 'Female', 293333.3),
 ('55-65', 'Male', -769000.0),
 ('underage', 'Male', None)]